
### RKHS - Reproducing Kernel Hilbert Space Method
Przestrzeń Hilberta $H\subset C(J)$ z iloczynem skalarnym (inner product) $\langle u,v\rangle$ ma **jądro reprodukujące** $K=\{k_x; \; x\in J\}$ 
jeśli 
$$u(x)=\langle u,k_x\rangle\qquad\text{dla wszystkich }\;u\in H, x\in J.$$

 If the domain $J=[a,b]$ is any finite closed interval,
 an arbitrary *reference point* $\xi\in [a,b]$ gives rise to a particular inner product
\begin{align}
\langle f,g\rangle_{m,\xi}=\sum_{i<m}f^{(i)}(\xi)g^{(i)}(\xi)+ \int_a^b f^{(m)}(t) g^{(m)}(t)dt
\end{align}
in the Sobolev space $H^m[a,b],$ which is well-known to be equivalent to \eqref{wz:skalar:m}, in the sense that the respective norms are equivalent. 


Given an arbitrary positive integer $m,$ we shall denote by $k=k^m\colon [0,\infty)\times[0,\infty)\to\R$  the reproducing kernel in $H^m[0,T],$ for any $T>0,$ discovered by Gorbunov and Petrishchev in 2003 \cite{GorbPetr}. The   explicit formula for the kernel is
\begin{align}
k^m(x,t)=\left\{\begin{array}{ll}
\sum_{i=0}^{m-1}\left(\frac{x^i}{i!}+(-1)^{m-1-i}\frac{x^{2m-1-i}}{(2m-1-i)!}
\right)\frac{t^i}{i!} & \text{if }0\leq x\leq t\\
k^m(t,x) & \text{if }0\leq t\leq x,
\end{array}\right.\end{align}  
for $x,t\geq 0,$ and the identity $\langle f,k_x\rangle_{m,0}=f(x)$ for $f\in H^m[0,T]$ and any $x$ is simply a classical Taylor expansion formula. If the reference point $0$ is no longer one of the endpoints, the associated reproducing kernel becomes a bit more complicated.

In [1]:
import numpy as np
import abc
from builtins import int
from typing import Sequence

from numpy import ndarray


class RK:
    def __init__(self, deg: int, ref: float = 0):
        self._deg = deg
        self._ref = ref

    def Dk(self, p, q: int, x, t: float) -> float:
        n = 2 * self._deg - p - q
        if n <= 0:
            return 0
        if x < 0 and t < 0:
            if (p + q) % 2 == 0:
                return self.Dk(p, q, -x, -t)
            else:
                return -self.Dk(p, q, -x, -t)
        if t < x:
            return self.Dk(q, p, t, x)
        Dk = 1
        if p >= self._deg:  # ±(t-x)^(2deg-p-q-1)/(2deg-p-q-1)!
            if x < 0:
                return 0
            t_x = t - x
            for i in range(1, n):
                Dk = Dk * t_x / i
            if (p - self._deg) % 2 != 0:
                Dk = -Dk
            return Dk
        # now  p < deg,  Dk = 1
        xt, x_t = x * t, 0
        if t > 0:
            x_t = x / t
        if x > 0:
            for i in range(1, self._deg - q):
                Dk = 1 - Dk * x_t * i / (n - i)
            Dk = 1 + Dk * x / (self._deg - p)
        for i in range(1, min(self._deg - p, self._deg - q)):
            Dk = 1 + Dk * xt / (self._deg - p - i) / (self._deg - q - i)
        # and the outer factor
        for i in range(p - q):  # *t^(p-q)/(p-q)!
            Dk = Dk * t / (i + 1)  # if (p>=q)
        for i in range(q - p):  # *x^(q-p)/(q-p)!
            Dk = Dk * x / (i + 1)  # if (p<q)
        return Dk

    def dk(self, p, q: int, x, t: float) -> float:
        return self.Dk(p, q, x - self._ref, t - self._ref)


def main():
    k5 = RK(5)  # k3.dk(2, 1, 0, 1);
    for i in range(10):
        for j in range(10):
            print(f'{i:2d},{j:2d},{k5.dk(2, 1, i * .1, j * .1):7.4f}', sep=': ', end='; ')
        print('\n')

main()

 0, 0, 0.0000;  0, 1, 0.1000;  0, 2, 0.2000;  0, 3, 0.3000;  0, 4, 0.4000;  0, 5, 0.5000;  0, 6, 0.6000;  0, 7, 0.7000;  0, 8, 0.8000;  0, 9, 0.9000; 

 1, 0, 0.0000;  1, 1, 0.1005;  1, 2, 0.2020;  1, 3, 0.3045;  1, 4, 0.4081;  1, 5, 0.5126;  1, 6, 0.6182;  1, 7, 0.7248;  1, 8, 0.8324;  1, 9, 0.9411; 

 2, 0, 0.0000;  2, 1, 0.1010;  2, 2, 0.2040;  2, 3, 0.3091;  2, 4, 0.4162;  2, 5, 0.5254;  2, 6, 0.6368;  2, 7, 0.7502;  2, 8, 0.8658;  2, 9, 0.9836; 

 3, 0, 0.0000;  3, 1, 0.1015;  3, 2, 0.2061;  3, 3, 0.3137;  3, 4, 0.4245;  3, 5, 0.5385;  3, 6, 0.6557;  3, 7, 0.7763;  3, 8, 0.9001;  3, 9, 1.0274; 

 4, 0, 0.0000;  4, 1, 0.1020;  4, 2, 0.2081;  4, 3, 0.3184;  4, 4, 0.4329;  4, 5, 0.5518;  4, 6, 0.6751;  4, 7, 0.8030;  4, 8, 0.9355;  4, 9, 1.0727; 

 5, 0, 0.0000;  5, 1, 0.1025;  5, 2, 0.2102;  5, 3, 0.3231;  5, 4, 0.4414;  5, 5, 0.5653;  5, 6, 0.6949;  5, 7, 0.8304;  5, 8, 0.9718;  5, 9, 1.1194; 

 6, 0, 0.0000;  6, 1, 0.1030;  6, 2, 0.2123;  6, 3, 0.3279;  6, 4, 0.4501;  6, 5, 0.5791